## HW2

> using ResNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# see more data augmentation https://pytorch.org/vision/stable/transforms.html
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)
train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)]) # calculte yourself

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, std)]) # calculte yourself

batch_size = 64
num_classes = 100    # check

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class my_CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # self.pre = nn.Sequential(
        #     nn.Conv2d(3, batch_size, 3, padding=1),
        #     nn.BatchNorm2d(64),
        #     nn.ReLU(inplace=True)
        # )

        self.conv_block = nn.Sequential(
            # unit 1
            nn.Conv2d(3,batch_size,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(3,batch_size,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2,padding=(1,1)),

            #unit 2
            nn.Conv2d(batch_size,batch_size*2,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(batch_size,batch_size*2,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2,padding=(1,1)),

            #unit 3
            nn.Conv2d(batch_size*2,batch_size*4,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(batch_size*2,batch_size*4,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2,padding=(1,1)),

            #unit 4
            nn.Conv2d(batch_size*4,batch_size*8,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(batch_size*4,batch_size*8,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2,padding=(1,1)),

            #unit 2
            nn.Conv2d(batch_size*8,batch_size*8,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(batch_size*8,batch_size*8,kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2,padding=(1,1))
        )

        self.dense_block = nn.Sequential(
            nn.Linear(batch_size*8, batch_size*4),
            nn.ReLU(),
            nn.Linear(batch_size*4, batch_size*2),
            nn.ReLU(),
            nn.Linear(batch_size*2, 100)
        )

    def forward(self, x) :
        # x = self.pre(x)
        conv_out = self.conv_block(x)
        res = conv_out.view(conv_out.size(0), -1) # view(-1) same as flatten
        out = self.dense_block(res)
        return out


In [ ]:
# pick one

# 1. model defined by yourself
# model = my_CNN()        
   
# 2. off-the-shelf model
# see https://pytorch.org/vision/stable/models.html
# nn.Linear https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
model = models.resnet50(pretrained=True) 
model.fc = torch.nn.Linear(2048, num_classes)
model.conv1 = nn.Conv2d(3,64,kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
model.maxpool = nn.Identity()

model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=1e-4)

In [ ]:
total_epoch = 200
print_per_iteration = 100
save_path = '/content/drive/MyDrive/Colab Notebooks/ML_HW2/myCNN_test/modify_ResNet.pth'

for epoch in range(total_epoch):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # print(inputs)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if (i+1) % print_per_iteration == 0:    # print every 2000 mini-batches
            print(f'[ep {epoch + 1}][{i + 1:5d}/{len(trainloader):5d}] loss: {loss.item():.3f}')
    torch.save(model, save_path)

[ep 1][  100/  782] loss: 4.602
[ep 1][  200/  782] loss: 4.621
[ep 1][  300/  782] loss: 4.510
[ep 1][  400/  782] loss: 4.556
[ep 1][  500/  782] loss: 4.486
[ep 1][  600/  782] loss: 4.458
[ep 1][  700/  782] loss: 4.355
[ep 2][  100/  782] loss: 4.163
[ep 2][  200/  782] loss: 4.177
[ep 2][  300/  782] loss: 4.063
[ep 2][  400/  782] loss: 4.030
[ep 2][  500/  782] loss: 4.074
[ep 2][  600/  782] loss: 3.864
[ep 2][  700/  782] loss: 3.842
[ep 3][  100/  782] loss: 3.793
[ep 3][  200/  782] loss: 3.844
[ep 3][  300/  782] loss: 3.763
[ep 3][  400/  782] loss: 3.533
[ep 3][  500/  782] loss: 3.697
[ep 3][  600/  782] loss: 3.790
[ep 3][  700/  782] loss: 3.438
[ep 4][  100/  782] loss: 3.297
[ep 4][  200/  782] loss: 3.284
[ep 4][  300/  782] loss: 3.244
[ep 4][  400/  782] loss: 3.250
[ep 4][  500/  782] loss: 3.212
[ep 4][  600/  782] loss: 3.034
[ep 4][  700/  782] loss: 3.145
[ep 5][  100/  782] loss: 2.997
[ep 5][  200/  782] loss: 2.768
[ep 5][  300/  782] loss: 2.703
[ep 5][ 

In [ ]:
# load trained model
# model = torch.load("./model.pth")
# model.to(device)

# fixed testing process
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Accuracy of the network on the 10000 test images: 72.71 %


Accuracy of the network on the 10000 test images: 60.19 %

In [ ]:
# model = models.mobilenet_v3_large()
# torch.save(model, "./model.pth")

# see size of saved model
! du -h model.pth

du: cannot access 'model.pth': No such file or directory
